# Exercise 6 – Linking Opportunity Index & ACE Proxy

In this exercise, you will examine how community opportunity factors relate to a proxy measure of **Adverse Childhood Experiences (ACEs)** using census tracts in New Jersey. You’ll build choropleth maps, compute simple composite indices, and explore correlations between measures of opportunity and a county‑level ACE proxy.

A **choropleth map** is a thematic map where each geographic area is shaded according to a data value. By colouring tracts based on the child opportunity indicators, you can quickly identify spatial patterns – such as clusters of low opportunity around certain cities or counties. To make values comparable, we’ll standardize each indicator and map **z‑scores** (standard deviations) rather than raw values. All maps use quantile breaks (equal‑sized groups) and a shared legend indicating that higher z‑scores represent better opportunity.

**Data sources**

- **Child Opportunity Index (COI) 2.0** – tract‑level indicators that measure educational, environmental and socio‑economic opportunities for children. The raw indicators we will use (all oriented so higher values mean better opportunity) are:
  - `ED_ECENROL` – early childhood (age 3–4) enrollment rate.
  - `ED_HSGRAD` – high school on‑time graduation rate.
  - `ED_MATH` – third‑grade math proficiency (NAEP scale).
  - `HE_GREEN` – greenspace/park access index (treated here as a community factor).
  - `SE_EMPRAT` – employment rate among working‑age adults.

Data Source: diversitydatakids.org. (2020). Child Opportunity Index 2.0 census tract data. Brandeis University, The Heller School for Social Policy and Management. Retrieved from https://www.diversitydatakids.org/research-library/child-opportunity-index-20-census-tract-data 

- **Child welfare ACE proxy** – county‑level counts of child welfare cases (`cw_cases_count`) and under‑18 population (`under18_pop`) from the NJ Child Welfare Data Hub and the ACS. We derive `cw_cases_per_1000` (cases per 1 000 children) and its z‑score `cw_cases_per_1000_Z`. This serves as a proxy for ACE prevalence. **Important:** this value is identical for all tracts in a county, so any correlation with tract‑level indicators is driven by differences *between counties* rather than within them.

Data source: New Jersey State Policy Lab. (n.d.). Child abuse and neglect dashboard. Rutgers University. Retrieved from https://njchilddata.rutgers.edu/portal/child-abuse-neglect

The dataset `merged_data_plus_aceproxy_rate.geojson` already merges these sources and includes 2010 tract geometries. Before mapping and analysis we will:

1. **Standardize** each COI indicator to a z‑score (subtract the NJ mean and divide by the standard deviation). Standardizing centres each variable at zero and puts different indicators on the same scale.  
2. **Compute composite indices**: an education mini‑COI (`COI_edu_mini`) averaging the three education z‑scores, and an overall mini‑COI (`COI_mini`) averaging all five z‑scores.  
3. **Create maps and plots**: make choropleth maps for your chosen variable (z‑score) and the composites, then create scatterplots comparing your variable and the composites to `cw_cases_per_1000_Z`, computing Pearson correlations.

Each student in your group will pick one of the raw COI variables (two from the Education domain and two from the Social/Economic domain) and follow the same workflow for that variable. Change the `CASE_VAR` in the configuration cell below when it’s your turn to run the notebook. You’ll save your figures into a group‑specific folder.


## Configuration

Set up a few variables that control your analysis.  
- **GROUP_ID:** used to name your output files.  
- **CASE_VAR:** the COI variable you’ve been assigned to analyse (`ED_ECENROL`, `ED_HSGRAD`, `ED_MATH`, `HE_GREEN`, or `SE_EMPRAT`).  
- **ED_VARS** and **SE_VARS:** lists of the education and social/economic variables.  
- **EXPORT_DIR:** the folder where your maps and plots will be saved.  
- **MAP_QUANTILES:** number of quantile classes for choropleth maps.  
- **SEED:** random seed for reproducibility in any stochastic steps (e.g. plotting jitter).

Edit `CASE_VAR` (and optionally `GROUP_ID`) before running the notebook for your assigned variable.


In [ ]:
# Working in Codespaces? Skip this cell.
# Working in VS Code Jupyter notebook? Uncomment and run the following lines if you need to install these packages.

# %pip install "mapclassify>=2.4.0"
# %pip install folium

In [ ]:
# Run this cell
import geopandas as gpd
import pandas as pd
import numpy as np
import os

# Configuration: change CASE_VAR to your assigned variable when running your section
GROUP_ID = "G07"  # Group identifier used in filenames

# Define sets of variables for convenience
actionvars_edu = ["ED_ECENROL", "ED_HSGRAD", "ED_MATH"]  # Education domain raw variables
actionvars_se = ["HE_GREEN", "SE_EMPRAT"]  # Social/Economic domain raw variables

# Combine names for full list of variables
ED_VARS = actionvars_edu
SE_VARS = actionvars_se

EXPORT_DIR = "figs_ex6_" + GROUP_ID  # Directory for saving figures
MAP_QUANTILES = 5  # Number of quantile classes for choropleth
SEED = 4242  # Random seed for reproducibility


### Group assignments

Enter your assigned variable in the code cell below (set CASE_VAR = "..." and run).

| Member   | Variable   | Description |
|----------|------------|-------------|
| Member 1 | `ED_HSGRAD` | high school on‑time graduation rate |
| Member 2 | `ED_MATH`   | third‑grade math proficiency (NAEP scale) |
| Member 3 | `SE_EMPRAT` | employment rate among working‑age adults |
| Member 4 | `HE_GREEN`  | greenspace/park access index (treated here as a community factor) |

Reminder: In the code cell below, set CASE_VAR to your assigned variable before running (e.g., CASE_VAR = "ED_HSGRAD").

In [ ]:
CASE_VAR =   # <-- set to your assigned variable


## Load Data & Quality Checks

We load the GeoJSON file containing New Jersey census tracts along with the COI indicators and ACE proxy. It’s important to verify that the data are well‑formed before analysis:

- **CRS check:** ensure the coordinate reference system is WGS 84 (EPSG: 4326).  
- **Unique identifiers:** each census tract is identified by its `GEOID`. Confirm that there are no duplicates.  
- **County broadcast check:** fields derived from the child welfare data (`cw_cases_count`, `cw_cases_per_1000`) should be constant for all tracts within the same county.  
- **Missing values:** identify and drop tracts with missing values in any of the variables you will analyse.  
- **Zero population:** drop any tracts with zero children under 18 so the ACE proxy isn’t undefined.

Run the cell below to load the data and perform these checks. It will also print how many tracts are dropped due to missing data or zero population.


In [ ]:
# Load the merged GeoJSON data
# Make sure this file exists in the working directory or adjust the path accordingly
gdf = gpd.read_file("merged_data_plus_aceproxy_rate.geojson")

# Print basic info
print("Total tracts loaded:", len(gdf))

# Check coordinate system
if gdf.crs is not None:
    try:
        epsg = gdf.crs.to_epsg()
        print("CRS:", gdf.crs)
        print("CRS EPSG code:", epsg)
        if epsg != 4326:
            raise ValueError("Data is not in EPSG:4326 as expected!")
    except Exception as e:
        print("CRS to_epsg() not available or not numeric. Reported CRS:", gdf.crs)

# Check that GEOIDs are unique
unique_ids = gdf["GEOID"].nunique()
if unique_ids != len(gdf):
    raise ValueError(f"Expected one row per tract, but found {unique_ids} unique GEOIDs for {len(gdf)} rows.")

# Verify county broadcast fields are constant within each county
for col in ["cw_cases_count", "cw_cases_per_1000"]:
    if col in gdf.columns:
        vals_per_county = gdf.groupby("county_fips")[col].nunique()
        if vals_per_county.max() != 1:
            problematic = vals_per_county[vals_per_county > 1]
            print(f"Warning: {col} has differing values within counties: {problematic.index.tolist()}")
        else:
            print(f"{col} is constant within each county (broadcast confirmed).")

# Drop tracts with no children (under18_pop <= 0)
if "under18_pop" in gdf.columns:
    no_kids = gdf[gdf["under18_pop"] <= 0]
    if len(no_kids) > 0:
        print(f"Dropping {len(no_kids)} tracts with no children (under18_pop <= 0).")
        gdf = gdf[gdf["under18_pop"] > 0].copy()

# Drop rows with missing values in variables of interest
key_vars = ED_VARS + SE_VARS + ["cw_cases_count", "cw_cases_per_1000"]
missing_report = {var: int(gdf[var].isna().sum()) for var in key_vars if var in gdf.columns}
missing_any = any(count > 0 for count in missing_report.values())
if missing_any:
    print("Missing values detected:", missing_report)
    before_len = len(gdf)
    gdf = gdf.dropna(subset=[var for var in key_vars if var in gdf.columns])
    after_len = len(gdf)
    print(f"Dropped {before_len - after_len} tracts due to missing data in key variables.")
else:
    print("No missing values detected in key variables.")

print("Data ready with", len(gdf), "tracts after cleaning.")


## Standardizing Variables

To compare indicators measured on different scales, convert each to a z‑score: subtract the mean and divide by the standard deviation across all NJ tracts. The helper function `zify()` adds a `_<var>_Z` column to the GeoDataFrame if it doesn’t already exist. Missing values remain `NaN` so they don’t influence the mean or standard deviation.


In [ ]:
def zify(df, var):
    """Compute and add a z‑score column for `var` in `df` (new column named `<var>_Z`).

    Parameters
    ----------
    df : pandas.DataFrame or geopandas.GeoDataFrame
        The data structure to which the z‑score column will be added.
    var : str
        Name of the variable to standardize.

    Returns
    -------
    str
        The name of the z‑score column.
    """
    z_col = var + "_Z"
    if z_col in df.columns:
        return z_col
    series = df[var]
    mean_val = series.mean(skipna=True)
    std_val = series.std(skipna=True)
    if std_val is None or std_val == 0:
        raise ValueError(f"Standard deviation for {var} is zero or undefined; cannot compute z‑score.")
    df[z_col] = (series - mean_val) / std_val
    return z_col


### Standardize Variables & Build Mini‑COI

Run the next cell to create z‑score versions of all COI indicators listed in `ED_VARS` and `SE_VARS`, as well as the ACE proxy rate. Then compute two simple composite indices:

- `COI_edu_mini` – the mean of the education z‑scores (`ED_ECENROL_Z`, `ED_HSGRAD_Z`, `ED_MATH_Z`).
- `COI_SE_mini` – the mean of the social/economic z‑scores (`HE_GREEN_Z` and `SE_EMPRAT_Z`).

These "mini COI" measures approximate the official COI domain scores by equally weighting each standardized indicator within their respective domains. We also save a small JSON file documenting exactly which z‑score columns were averaged in each composite for reproducibility.

In [ ]:
# Standardize COI variables
for var in ED_VARS + SE_VARS:
    if var in gdf.columns:
        zify(gdf, var)

# Standardize the ACE proxy rate
if "cw_cases_per_1000" in gdf.columns:
    zify(gdf, "cw_cases_per_1000")

# Compute mini COI composites
edu_components = [f"{v}_Z" for v in ED_VARS]
se_components = [f"{v}_Z" for v in SE_VARS]

# Check that all z‑score columns exist
for col in edu_components + se_components:
    if col not in gdf.columns:
        raise KeyError(f"Expected z‑score column {col} missing; ensure variables were standardized.")

# Education mini index
gdf["COI_edu_mini"] = gdf[edu_components].mean(axis=1)

# Social/Economic mini index
gdf["COI_SE_mini"] = gdf[se_components].mean(axis=1)

# Save recipe for reproducibility
mini_coi_recipe = {
    "COI_edu_mini": edu_components,
    "COI_SE_mini": se_components,
    "note": "All components are oriented so higher = better before averaging."
}

os.makedirs(EXPORT_DIR, exist_ok=True)

import json
with open(os.path.join(EXPORT_DIR, f"{GROUP_ID}_mini_coi_recipe.json"), "w") as f:
    json.dump(mini_coi_recipe, f, indent=2)

print("Composite indices computed and recipe saved to", os.path.join(EXPORT_DIR, f"{GROUP_ID}_mini_coi_recipe.json"))


## Choropleth Maps

To visualize spatial patterns, we'll create choropleth maps for:

- Your selected variable's z‑score (`<CASE_VAR>_Z`).  
- The education mini‑COI (`COI_edu_mini`).  
- The social/economic mini‑COI (`COI_SE_mini`).  
- *(Optional)* the raw variable and the ACE proxy z‑score (`cw_cases_per_1000_Z`).

All maps use five quantile classes (defined by `MAP_QUANTILES`) and share a consistent legend that reads "z‑score (SD): higher = better". County boundaries are drawn to emphasize that the ACE proxy is constant within each county. Running the following cell will produce and save the maps to the directory specified by `EXPORT_DIR`. Adjust the colour map or figure size if desired.

In [ ]:
import folium
import numpy as np
import os

# Ensure export dir exists
os.makedirs(EXPORT_DIR, exist_ok=True)

# We’ll use the full GeoJSON from gdf; Folium likes WGS84 (EPSG:4326), which you already have.
# Small helper to compute quantile bins robustly (no mapclassify needed)
def _quantile_bins(series, k=5):
    # dropna and compute quantiles; make sure edges are strictly increasing
    q = np.linspace(0, 1, k + 1)
    vals = np.nanquantile(series.dropna(), q)
    # Ensure uniqueness (np.nanquantile can return repeats with discrete data)
    eps = 1e-9
    for i in range(1, len(vals)):
        if vals[i] <= vals[i-1]:
            vals[i] = vals[i-1] + eps
    return list(vals)

def _nice_center(gdf_):
    # rough center of New Jersey; fallback to lat/lon mean if geometry bounds fail
    try:
        minx, miny, maxx, maxy = gdf_.total_bounds
        return [(miny+maxy)/2, (minx+maxx)/2]
    except Exception:
        return [40.06, -74.4]

def interactive_choropleth(column, title, html_filename, bins=None, fill_color="Blues",
                           legend_title="z-score (SD) [higher = better]"):
    # Compute bins if not provided
    if bins is None:
        bins = _quantile_bins(gdf[column], k=MAP_QUANTILES)

    # Base map (students can zoom/pan)
    m = folium.Map(location=_nice_center(gdf), zoom_start=8, tiles="cartodbpositron")

    # Choropleth layer
    chor = folium.Choropleth(
        geo_data=gdf.to_json(),            # full tract geometries + properties
        data=gdf,                          # DataFrame with values
        columns=["GEOID", column],         # bind by GEOID
        key_on="feature.properties.GEOID",
        fill_color=fill_color,
        fill_opacity=0.85,
        line_opacity=0.2,
        line_color="gray",
        bins=bins,
        nan_fill_opacity=0.0,
        legend_name=legend_title,          # appears as legend title
        smooth_factor=0.5,
        name=title
    )
    chor.add_to(m)

 # Hover tooltip with tract + county + value (keep geometry!)
    try:
        tooltip_gdf = gdf[["GEOID", "county_fips", column, gdf.geometry.name]].rename(columns={column: "value"})
    except KeyError:
        raise KeyError(f"Column '{column}' not found in GeoDataFrame.")
    tooltip = folium.features.GeoJson(
        data=tooltip_gdf,  # pass a GeoDataFrame (with geometry), not a JSON string
        name="tooltip",
        style_function=lambda x: {"color": "transparent", "fillOpacity": 0},
        highlight_function=lambda x: {"weight": 2, "color": "black", "fillOpacity": 0.1},
        tooltip=folium.features.GeoJsonTooltip(
            fields=["GEOID", "county_fips", "value"],
            aliases=["GEOID", "County FIPS", f"{column}"],
            localize=True,
            sticky=False,
            labels=True,
        ),
    )
    tooltip.add_to(m)

    # Optional: label county FIPS at centroids (lightweight text annotations)
    try:
        counties = gdf.dissolve(by="county_fips")
        counties["cx"] = counties.geometry.centroid.y
        counties["cy"] = counties.geometry.centroid.x
        for fips, row in counties.iterrows():
            folium.map.Marker(
                [row["cx"], row["cy"]],
                icon=folium.DivIcon(
                    html=f'<div style="font-size:10px;color:black;opacity:0.8">{fips}</div>'
                ),
            ).add_to(m)
    except Exception:
        pass

    folium.LayerControl(collapsed=True).add_to(m)
    out_path = os.path.join(EXPORT_DIR, html_filename)
    m.save(out_path)
    print("Saved interactive map →", out_path)

# === BUILD INTERACTIVE MAPS ===
given_var = CASE_VAR
z_var = f"{CASE_VAR}_Z"

# CASE var (Z)
interactive_choropleth(
    column=z_var,
    title=f"{given_var} (Z-score) by Tract",
    html_filename=f"{GROUP_ID}_{given_var}_Z_map.html",
    fill_color="Blues",
    legend_title="z-score (SD) [higher = better]"
)

# Education mini-COI
interactive_choropleth(
    column="COI_edu_mini",
    title="COI_edu_mini (Education Opportunity Index)",
    html_filename=f"{GROUP_ID}_COI_edu_mini_map.html",
    fill_color="Blues",
    legend_title="z-score (SD) [higher = better]"
)

# Social/Economic mini-COI
interactive_choropleth(
    column="COI_SE_mini",
    title="COI_SE_mini (Social/Economic Opportunity Index)",
    html_filename=f"{GROUP_ID}_COI_SE_mini_map.html",
    fill_color="Blues",
    legend_title="z-score (SD) [higher = better]"
)

# Optional: ACE proxy map (county-broadcast; different palette and legend title)
# interactive_choropleth(
#     column="cw_cases_per_1000_Z",
#     title="Child Welfare Case Rate (Z) by Tract (County-level)",
#     html_filename=f"{GROUP_ID}_ACEproxy_map.html",
#     fill_color="OrRd",
#     legend_title="ACE proxy z-score (SD) [higher = more cases]"
# )


## ✅ Complete Table 1 (use your assigned variable)

### 1) Pick one tract and report its z-score (for item 1 in Table 1)
Open your group’s **interactive choropleth HTML map** (saved in your `EXPORT_DIR`).  
**Zoom in** on a neighborhood and **click one census tract.**  
The popup will show:

- **GEOID** (the 11-digit census-tract ID)  
- **County FIPS** code  
- your variable’s **z-score** (for example: `ED_HSGRAD_Z = −1.706`)  

Record that GEOID and z-score. Interpret it in plain language in ≤ 280 characters (± SD from NJ mean + approximate percentile).  

> Example popup (what you’ll see when you click a tract):  
> ![Example tract popup with GEOID and z-score](zscoredemo.png)

---

### 2) Use these short primers to answer the COI “composition & calculation” prompts (questions 2 and 3 in Table 1)
These summarize how the **official Child Opportunity Index (COI) 2.0** builds its domain composites.  
Our classroom mini-COIs are simplified teaching versions—use these notes for comparison.

#### COI 2.0 in one paragraph
COI 2.0 measures child-relevant neighborhood opportunity using **29 indicators** grouped into **three domains**—**Education**, **Health & Environment**, and **Social & Economic**—for all U.S. census tracts.  
Each indicator is **z-scored** (2010 baseline) and direction-aligned so that higher = more opportunity.  
Indicators are then **weighted** (not equally) based on their average association with independent health and economic outcomes.  
Weighted indicators form domain scores; the overall COI aggregates those domains with the same evidence-based weighting.  
Child Opportunity Levels (Very Low → Very High) correspond to percentile groups of these standardized domain / overall scores.

#### Education Domain – What it Measures & How it’s Combined
- **Indicators:** early-childhood education access & enrollment, 3rd-grade reading and math proficiency, on-time high-school graduation, AP course participation, nearby college enrollment, school poverty (reversed), teacher experience (reversed), and adult educational attainment.  
- **Computation:** each indicator → z-score (2010), direction reversed where needed, then assigned a data-driven weight according to its predictive strength; weighted indicators sum to the Education domain score.  
- **Our mini version:**  
  `COI_edu_mini = mean(ED_ECENROL_Z, ED_HSGRAD_Z, ED_MATH_Z)` (equal weights; higher = better).

#### Social & Economic Domain – What it Measures & How it’s Combined
- **Indicators:** economic resources and family structure factors—employment rate, long commute share (reversed), single-parent household share (reversed), plus an **Economic Resource Index** combining poverty, public assistance, homeownership, high-skill employment, and median household income via principal components analysis (PCA).  
- **Computation:** z-score each indicator, align direction, apply empirical weights from predictive models, and sum to the Social & Economic domain score.  
- **Our mini COI version**
  `COI_SE_mini = mean(HE_GREEN_Z, SE_EMPRAT_Z)` (equal weights; higher = better).  
  *Note:* The official Social & Economic domain includes more socioeconomic measures (poverty, income, housing, family structure) and weighted aggregation for greater validity.
---

### Learn more (official source)
**Child Opportunity Index 2.0 – Technical Documentation** (education & social/economic indicator lists, standardization, PCA economic resource index, weighting and aggregation details):  
https://www.diversitydatakids.org/sites/default/files/2020-02/ddk_coi2.0_technical_documentation_20200212.pdf

*Footnote:* `COI_edu_mini` = mean(ED_ECENROL_Z, ED_HSGRAD_Z, ED_MATH_Z); `COI_SE_mini` = mean(HE_GREEN_Z, SE_EMPRAT_Z). Higher = better.


## Scatterplots & Correlation

In Part 2 you will explore the relationship between opportunity indicators and the ACE proxy. The expectation is a **negative correlation** – higher opportunity should be associated with fewer child welfare cases. Each student will produce:

- **Scatter A:** your chosen variable’s z‑score (`<CASE_VAR>_Z`) against `cw_cases_per_1000_Z`, including a fitted linear regression line and labels for Pearson *r* and *r²*.
- **Scatter B:** the education composite (`COI_edu_mini`) against `cw_cases_per_1000_Z` (same for everyone; generate it once).
- **Scatter C:** the overall composite (`COI_mini`) against `cw_cases_per_1000_Z`.

Because the ACE proxy is constant within each county, the correlation you observe is entirely due to differences **between counties**. You can compute a between‑county correlation by aggregating tracts to county means. The code below generates the scatterplots, annotates the correlation coefficients, and optionally calculates the county‑level correlation for your variable.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Ensure reproducibility
np.random.seed(SEED)

# Prepare variables
x_var = CASE_VAR + "_Z"
y_var = "cw_cases_per_1000_Z"

# Function to create a scatter plot with regression line and correlation annotation
def scatter_with_corr(x_data, y_data, xlabel, ylabel, title, filename):
    fig, ax = plt.subplots(figsize=(5.5, 5.5))
    sns.regplot(x=x_data, y=y_data, ci=None, scatter_kws={"alpha": 0.5}, line_kws={"color": "red"}, ax=ax)
    # Compute correlation
    mask = ~x_data.isna() & ~y_data.isna()
    r = np.corrcoef(x_data[mask], y_data[mask])[0, 1]
    r2 = r ** 2
    ax.text(0.97, 0.03, f"r = {r:.2f}, r² = {r2:.2f}", transform=ax.transAxes,
            horizontalalignment='right', verticalalignment='bottom', fontsize=11,
            bbox=dict(boxstyle="round,pad=0.2", fc="white", alpha=0.8))
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    plt.tight_layout()
    plt.savefig(os.path.join(EXPORT_DIR, filename), dpi=150)
    plt.close(fig)
    return r

# Scatter A: CASE_VAR_Z vs ACE proxy
r_case = scatter_with_corr(gdf[x_var], gdf[y_var], f"{CASE_VAR} (Z-score)", "Child Welfare Cases per 1000 (Z-score)",
                          f"{CASE_VAR}_Z vs Child Welfare Case Rate (Z)", f"{GROUP_ID}_scatter_{CASE_VAR}.png")
print(f"Scatter plot saved for {CASE_VAR}_Z vs {y_var} with r = {r_case:.2f}.")

# Scatter B: COI_edu_mini vs ACE proxy
r_edu = scatter_with_corr(gdf["COI_edu_mini"], gdf[y_var], "COI_edu_mini (Mean Education Z)",
                          "Child Welfare Cases per 1000 (Z-score)", "COI_edu_mini vs Child Welfare Case Rate (Z)",
                          f"{GROUP_ID}_scatter_COI_edu_mini.png")
print(f"Scatter plot saved for COI_edu_mini vs {y_var} with r = {r_edu:.2f}.")

# Scatter C: COI_SE_mini vs ACE proxy
r_se = scatter_with_corr(gdf["COI_SE_mini"], gdf[y_var], "COI_SE_mini (Mean Social/Economic Z)",
                         "Child Welfare Cases per 1000 (Z-score)", "COI_SE_mini vs Child Welfare Case Rate (Z)",
                         f"{GROUP_ID}_scatter_COI_SE_mini.png")
print(f"Scatter plot saved for COI_SE_mini vs {y_var} with r = {r_se:.2f}.")

# Optional: between-county correlation (aggregated by county)
county_means = gdf.groupby("county_fips")[[x_var, y_var]].mean()
if county_means[x_var].std() > 0:
    r_between = county_means[x_var].corr(county_means[y_var])
    print(f"Between-county correlation for {CASE_VAR}_Z: r_between = {r_between:.2f}")
else:
    print("No variation in the predictor across counties; cannot compute between-county correlation.")


### Optional: Compile Correlation Metrics

You can compile correlation metrics for all standardized variables and the composite indices, both at the tract level and aggregated by county. The following cell builds a simple DataFrame of correlations (*r* and *r²*) for each predictor against the ACE proxy, writes it to a CSV, and displays the first few rows. This is helpful for ranking variables by the strength of their association with ACE rates.


In [ ]:
# Compile correlation metrics across variables and composites
metrics = []

# List all z-score predictors
predictors = [f"{v}_Z" for v in ED_VARS + SE_VARS] + ["COI_edu_mini", "COI_SE_mini"]

for pred in predictors:
    # Tract-level correlation
    r_all = gdf[pred].corr(gdf["cw_cases_per_1000_Z"])
    metrics.append({
        "predictor": pred,
        "level": "tracts_all",
        "N": len(gdf),
        "r": round(r_all, 3),
        "r2": round(r_all**2, 3)
    })
    # County-level correlation (aggregate means)
    county_vals = gdf.groupby("county_fips")[[pred, "cw_cases_per_1000_Z"]].mean()
    r_cnty = county_vals[pred].corr(county_vals["cw_cases_per_1000_Z"])
    metrics.append({
        "predictor": pred,
        "level": "county_means",
        "N": len(county_vals),
        "r": round(r_cnty, 3) if r_cnty is not None else None,
        "r2": round(r_cnty**2, 3) if r_cnty is not None else None
    })

metrics_df = pd.DataFrame(metrics)
metrics_csv_path = os.path.join(EXPORT_DIR, f"{GROUP_ID}_metrics.csv")
metrics_df.to_csv(metrics_csv_path, index=False)
print("Saved correlation metrics to", metrics_csv_path)

# Display first few rows
metrics_df.head()


## Next Steps

- Complete remaining Working Tables in shared slides and group comparison questions!
- Follow submission directions on Canvas